In [86]:
from pathlib import Path
from openai import OpenAI
import os
import time
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import bibtexparser

In [90]:
#Initialize GPT client
client = OpenAI(
    api_key="sk-evR82XrLZswPmf23SwvEIZzqyL7Cg4SN2BAodZLtP1vQcwp7", 
    base_url="https://api.moonshot.cn/v1",
)
main_prompt = '''You are a GPT-based AI assistant focusing on Human-AI research. You will provide the user summaries of the paper they upload. Specifically, your response should contain the following aspects: (1) Context: Specify whether the study is focused on a specific industry, task or a broader, conceptual scope. (2) Research Question and Findings: Identify the main research question and summarise the key findings. (3) Classify Theme of Research into Human vs. AI (which focus more on comparing AI and human's performance in specific tasks) or Human + AI collaboration. (4) Classify the study method as one of the following: Conceptual/Case Study, Modeling or  Empirical Study. If the method is classified as empirical study, further determine whether the study is an experiment or a secondary data analysis. (5) Identify the primary contribution of the study, categorizing it as theoretical, managerial, or methodological. (6) Summarise what the study states about future research directions or the limitations of its findings.
Include the title of the paper at the beginning of the response.'''
answer_prompt = '''You are a GPT-based AI assistant focusing on Human-AI research. Answer two questions based on summary of a paper:
(1) What is the research category of the paper? Choose your answer from "Human + AI collaboration" or "Human vs AI".
(2) What is the method of the paper? Choose your answer from "Conceptual\Case Study", "Modeling", "Empirical Study(Experiment)" or "Empirical Study (Data Analysis)".
Use a comma to separate the answers. Include ONLY THE ANSWERS in your response!
'''

keyword_prompt = '''You are a GPT-based AI assistant focusing on Human-AI research. Your user wants you to summarize papers related to specific topics and research method. Luckily, both the method and the topics are limmited to a small range.
Possible methods inlclude "Empirical Study", "Modeling", and "Conceptual/Case Study". Possible topics include "Human and AI collaboration" and "Human vs AI".
Your task is to identify the desired method and topics given the user's input.
Include ONLY the method and the topic provided above in your response and use a comma to seperate them.
The user's input is as follows:

'''

review_prompt = '''You are a GPT-based AI assistant focusing on Human-AI research. You are given the summaries of a few papers on similar topics, and your task is to write a breif literature review according to them. 
Specifically, you should :
(1)Compile the summaries into coherent paragraphs that discuss how the research in these areas is connected, identifying common themes, trends, and potential future directions.
(2)In the last line of your response, provide the Titles of the papers, sperate them with commas. Strictly follow this direction in the last line!
The summeraries provided this time are about empirical studies on Human vs AI'''


In [88]:
#Initialize Database
conn = sqlite3.connect('HAI_database.db')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS HAI_Paper (
    id INTEGER PRIMARY KEY,
    file_path TEXT NOT NULL,
    method TEXT NOT NULL,
    category TEXT NOT NULL,
    summary TEXT NOT NULL
)
''')
#define functions for database operations, including creating, count, and filter

def insert_record(file_path, category, method, summary):
    cursor.execute('''
    INSERT INTO HAI_Paper (file_path, category, method, summary)
    VALUES (?, ?, ?, ?)
    ''', (file_path, category, method, summary))
    conn.commit()

def count_articles_by_category():
    cursor.execute('''
    SELECT method, COUNT(*)
    FROM HAI_paper
    GROUP BY method
    ''')
    return cursor.fetchall()

def filter_summaries(method, category):
    cursor.execute('''
    SELECT * FROM HAI_Paper
    WHERE (method LIKE ?)
    AND (category LIKE ?)
    ''', (f'%{method}%',f'%{category}%',))
    
    entries = cursor.fetchall()
    summaries = [f'{entry[4]}' for entry in entries]
    summary_for_review = ""
    for summary in summaries:
        summary_for_review += summary
    return summary_for_review



In [83]:
#Supporting Functions for generating citations
def bibtex_to_apa(bibtex_text):
    authors = bibtex_text['author'].replace(' and ', ', ')
    title = bibtex_text['title']
    journal = bibtex_text['journal']
    volume = bibtex_text['volume']
    number = bibtex_text['number']
    pages = bibtex_text['pages']
    year = bibtex_text['year']
    return f"{authors} ({year}). {title}. {journal}, {volume}({number}), {pages}."

def create_citation(title):
    # 打开谷歌学术
    driver = webdriver.Chrome()
    driver.get('https://scholar.google.com/')
    
    # 输入文章标题并搜索
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(title)
    search_box.send_keys(Keys.RETURN)
    
    time.sleep(2)  # 等待搜索结果加载
    
    # 点击第一个搜索结果的引用按钮
    driver.find_element(By.CLASS_NAME,'gs_or_cit.gs_nph').click()
    
    time.sleep(1)  # 等待引用弹窗加载
    
    # 获取BibTeX格式的引用
    bibtex_link = driver.find_element(By.XPATH, '//*[@id="gs_citi"]/a[contains(text(), "BibTeX")]')
    bibtex_link.click()
    
    time.sleep(1)  # 等待BibTeX页面加载
    
    bibtex_str = driver.find_element(By.XPATH, '/html/body/pre').text
    driver.quit()
    bib_database = bibtexparser.loads(bibtex_str)
    bibtex_text = bib_database.entries[0]
    apa_citation = bibtex_to_apa(bibtex_text)
    return apa_citation

In [ ]:
#Section 1, Function 1,Demo
file_object = client.files.create(file=Path("article/More than a bot.pdf"), purpose="file-extract") 
file_content = client.files.content(file_id=file_object.id).text
completion = client.chat.completions.create(
    model = "moonshot-v1-128k",
    messages = [
        {"role": "system", "content": main_prompt },
        {
        "role": "system",
        "content": file_content,
    },
        {"role": "user", "content": "Summarize the paper according to the request, and include the paper's title at the beginning"}
    ],
    temperature = 0.3,
)

summary = completion.choices[0].message.content
completiona = client.chat.completions.create(
model = "moonshot-v1-128k",
messages = [
    {"role": "system", "content": answer_prompt },
    {
    "role": "system",
    "content": summary,
    },
    {"role": "user", "content": "Answer the questions according to the request."}
],
temperature = 0.1, #make sure LLM answer according to the request
)
answer = completiona.choices[0].message.content
alist = answer.split(',')
category = alist[0]
method = alist[1]
print(summary,category,method)

**Title:** More Than a Bot? The Impact of Disclosing Human Involvement on Customer Interactions with Hybrid Service Agents

**Context:** This study focuses on the impact of disclosing human involvement in hybrid service agents, which are combinations of artificial intelligence (AI) and human employees behind a single interface, in online service encounters.

**Research Question and Findings:**
- **Research Question:** The main research question is whether disclosing human involvement in customer-hybrid service agent interactions affects customer communication behavior and employee workload.
- **Key Findings:** 
  1. Disclosing human involvement leads customers to adopt a more human-oriented communication style, characterized by longer, more complex, and more natural messages.
  2. This change in communication style increases employee workload because fewer customer requests can be handled automatically by the chatbot and must be delegated to a human.
  3. The effect of human involvemen

In [ ]:
#Section 1, Function 2, database built to save LLM tokens
paperfolderpath = "article/"
papers = os.listdir(paperfolderpath)
counter = 0
for paper in papers:
    counter += 1
    paper_path = os.path.join(paperfolderpath,paper)
    file_object = client.files.create(file=Path(paper_path), purpose="file-extract") 
    file_content = client.files.content(file_id=file_object.id).text
    completion = client.chat.completions.create(
    model = "moonshot-v1-128k",
    messages = [
        {"role": "system", "content": main_prompt },
        {
        "role": "system",
        "content": file_content,
    },
        {"role": "user", "content": "Summarize the paper according to the request, and include the paper's title at the beginning"}
    ],
    temperature = 0.3,
    )

    summary = completion.choices[0].message.content
    completiona = client.chat.completions.create(
    model = "moonshot-v1-128k",
    messages = [
        {"role": "system", "content": answer_prompt },
        {
        "role": "system",
        "content": summary,
    },
        {"role": "user", "content": "Answer the questions according to the request."}
    ],
    temperature = 0.1,
    )
    answer = completiona.choices[0].message.content
    category = answer.split(',')[0]
    method = answer.split(',')[1]
    insert_record(file_object.id,category,method,summary)
    print("Summary ",counter, " recorded")
for row in count_articles_by_category():
    print(f'Category: {row[0]}, Count: {row[1]}')


Category: Conceptual/Case Study, Count: 5
Category: Empirical Study(Data Analysis), Count: 5
Category: Empirical Study(Experiment), Count: 15
Category: Modeling, Count: 7


In [91]:
#Section 2
user_input = "please summarise empirical research about human VS AI"
completionk = client.chat.completions.create(
    model = "moonshot-v1-128k",
    messages = [
        {"role": "system", "content": keyword_prompt },
        {"role": "user", "content": user_input}
    ],
    temperature = 0.1,
    )
keywords = completionk.choices[0].message.content

method = keywords.split(',')[0].strip()
category = keywords.split(',')[1].strip()
summary_for_review = filter_summaries(method, category)

#print(summary)
completions = client.chat.completions.create(
    model = "moonshot-v1-128k",
    messages = [
        {"role": "system", "content": review_prompt },
        {
        "role": "system",
        "content": summary_for_review,
    },
        {"role": "user", "content": "Write the literature review according to the request,follow the directions about the last line strictly."}
    ],
    temperature = 0.3,
)
literature_review = completions.choices[0].message.content

lines = literature_review.split('\n')
last_line = lines[-1]
review_body = '\n'.join(lines[:-1])

review_body += '\nReference\n'
for title in last_line.split(','):
    review_body += create_citation(title)
    review_body += '\n'

print(review_body)

conn.close()

The field of human versus AI decision-making has been gaining significant attention, with researchers exploring various aspects of consumer reactions and behaviors when interacting with AI systems. Two recent empirical studies have shed light on this topic, providing valuable insights into how humans perceive and respond to decisions made by AI versus humans.

The first study, "Thumbs Up or Down: Consumer Reactions to Decisions by Algorithms Versus Humans," investigates consumer reactions to decisions made by algorithms versus humans across various industries, such as country club memberships, bank loan applications, and research participant pool selections. The researchers found that consumers exhibit less positive attitudes toward a company when receiving favorable decisions made by algorithms rather than humans. This negative reaction to algorithmic favorable decisions is not observed for unfavorable decisions, as consumers' reactions are similar regardless of whether a human or alg

In [93]:
with open("literature_review.txt", 'w') as f:
    f.writelines(review_body)
f.close()
